In [ ]:
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
from IPython.display import display
from time import gmtime, strftime
from sagemaker.predictor import csv_serializer

# Define IAM role
role = get_execution_role()
prefix = 'sagemaker/DEMO-xgboost-dm'
my_region = boto3.session.Session().region_name # set the region of the instance

# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
xgboost_container = sagemaker.image_uris.retrieve("xgboost", my_region, "latest")

print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + xgboost_container + " container for your SageMaker endpoint.")

In [ ]:
bucket_name = 'bucket-name' # <--- CHANGE THIS VARIABLE TO A UNIQUE NAME FOR YOUR BUCKET
ecs_access_key_id='ECS_ACCESS_KEY' # <--- Replace this variable with the Access Key ID
ecs_secret_access_key='ECS_SECRET_ACCESS_KEY' # <--- Replace this variable with the Secret Access Key
endpoint_url='https://1.2.3.4:5678' # <--- Replace this variable with the IP Address and Port number
s3 = boto3.resource(service_name='s3',aws_access_key_id=ecs_access_key_id,aws_secret_access_key=ecs_secret_access_key,endpoint_url=endpoint_url)
try:
    s3.create_bucket(Bucket=bucket_name)
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

In [ ]:
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('melb_data.csv', index=False, header=False)
s3_data=s3.Object(bucket_name,os.path.join(prefix, 'melb_data.csv')).upload_file('melb_data.csv')
s3_input_train = sagemaker.inputs.TrainingInput(s3_data)

In [ ]:
import pandas as pd

In [ ]:
# save filepath to variable for easier access
melbourne_file_path = '../input/melbourne-housing-snapshot/melb_data.csv'
# read the data and store data in DataFrame titled melbourne_data
melbourne_data = pd.read_csv(melbourne_file_path) 
# print a summary of the data in Melbourne data
melbourne_data.describe()